In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras

model = keras.models.load_model('Model_v2.keras')
GNIFTY_200D_DATA_PATH = '../Data/Past_200D_Data.csv'

In [1]:
from datetime import datetime, timedelta

now = datetime.now()
time_delta = timedelta(days=200)
start = now - time_delta
end = now

print(f"From: {start} to {end}")

From: 2025-04-19 21:48:33.057391 to 2025-11-05 21:48:33.057391


In [3]:
ticker = "^NSEI"
df_Nifty = yf.download(ticker, start, end)
# Calculate RSI (Relative Strength Index)
def calculate_rsi(data, window=14):
    diff = data.diff(1).dropna()
    gain = diff.mask(diff < 0, 0)
    loss = diff.mask(diff > 0, 0).abs()
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df_Nifty['RSI'] = calculate_rsi(df_Nifty['Close'])

# Calculate MACD (Moving Average Convergence Divergence)
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    short_ema = data.ewm(span=short_window, adjust=False).mean()
    long_ema = data.ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal_line = macd.ewm(span=signal_window, adjust=False).mean()
    return macd, signal_line

macd, signal_line = calculate_macd(df_Nifty['Close'])
df_Nifty['MACD'] = macd
df_Nifty['MACD_Signal'] = signal_line

# Calculate EMA 50
df_Nifty['EMA_50'] = df_Nifty['Close'].ewm(span=50, adjust=False).mean()

# Calculate SMA 20
df_Nifty['SMA_20'] = df_Nifty['Close'].rolling(window=20).mean()

# Calculate Stochastic RSI
def calculate_stoch_rsi(data, window=14, k_window=3, d_window=3):
    rsi = calculate_rsi(data, window)
    min_rsi = rsi.rolling(window=window).min()
    max_rsi = rsi.rolling(window=window).max()
    stoch_rsi = (rsi - min_rsi) / (max_rsi - min_rsi)
    k_line = stoch_rsi.rolling(window=k_window).mean()
    d_line = k_line.rolling(window=d_window).mean()
    return k_line, d_line

df_Nifty['StochRSI_K'], df_Nifty['StochRSI_D'] = calculate_stoch_rsi(df_Nifty['Close'])

df_Nifty.columns = [col for col, _ in df_Nifty.columns]
print(df_Nifty.columns)

C:\Users\Krish Patel\AppData\Local\Temp\ipykernel_19584\454345099.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_Nifty = yf.download(ticker, start, end)
[*********************100%***********************]  1 of 1 completed

Index(['Close', 'High', 'Low', 'Open', 'Volume', 'RSI', 'MACD', 'MACD_Signal',
       'EMA_50', 'SMA_20', 'StochRSI_K', 'StochRSI_D'],
      dtype='object')


In [4]:
df_Nifty

,Close,High,Low,Open,Volume,RSI,MACD,MACD_Signal,EMA_50,SMA_20,StochRSI_K,StochRSI_D
Date,,,,,,,,,,,,
2025-04-21,24125.550781,24189.550781,23903.650391,23949.150391,406100,NaN,0.000000,0.000000,24125.550781,NaN,NaN,NaN
2025-04-22,24167.250000,24242.599609,24072.000000,24185.400391,440900,NaN,3.326433,0.665287,24127.186045,NaN,NaN,NaN
2025-04-23,24328.949219,24359.300781,24119.949219,24357.599609,415000,NaN,18.793788,4.290987,24135.098326,NaN,NaN,NaN
2025-04-24,24246.699219,24347.849609,24216.150391,24277.900391,358800,NaN,24.136659,8.260121,24139.474832,NaN,NaN,NaN
2025-04-25,24039.349609,24365.449219,23847.849609,24289.000000,387700,NaN,11.506897,8.909476,24135.548352,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-10-29,26053.900391,26097.849609,25960.300781,25982.000000,321900,82.726544,268.728626,208.777421,25245.831645,25419.345215,0.837457,0.845284
2025-10-30,25877.849609,26032.050781,25845.250000,25984.400391,257400,72.026127,261.044734,219.230884,25270.616663,25482.682715,0.758621,0.805434
2025-10-31,25722.099609,25953.750000,25711.199219,25863.800781,334400,63.378740,239.625220,223.309751,25288.321877,25526.972656,0.617375,0.737818


In [5]:
df_Gnifty = pd.read_csv(GNIFTY_200D_DATA_PATH)
df_Gnifty.head()

,Date,Open,Close,Change(%),High,Low
0,04 Nov 2025,"25,898.00","25,712.00",-0.72%,"25,898.00","25,648.00"
1,03 Nov 2025,"25,832.50","25,897.50",0.25%,"25,938.50","25,784.50"
2,31 Oct 2025,"26,042.00","25,803.50",-0.92%,"26,113.00","25,803.50"
3,30 Oct 2025,"26,133.50","26,042.00",-0.35%,"26,197.50","25,992.00"
4,29 Oct 2025,"26,116.50","26,134.00",0.07%,"26,278.00","26,112.00"


In [6]:
df_Gnifty.rename(columns={
    'Close': 'GIFT_Close',
    'Open': 'GIFT_Open',
    'High': 'GIFT_High',
    'Low': 'GIFT_Low',
    'Vol.': 'GIFT_Volume',
    'Change %': 'GIFT_Change_Percent'
}, inplace=True)
df_Gnifty['Date'] = pd.to_datetime(df_Gnifty['Date'], format='%d %b %Y')
df_Gnifty.head()

,Date,GIFT_Open,GIFT_Close,Change(%),GIFT_High,GIFT_Low
0,2025-11-04,"25,898.00","25,712.00",-0.72%,"25,898.00","25,648.00"
1,2025-11-03,"25,832.50","25,897.50",0.25%,"25,938.50","25,784.50"
2,2025-10-31,"26,042.00","25,803.50",-0.92%,"26,113.00","25,803.50"
3,2025-10-30,"26,133.50","26,042.00",-0.35%,"26,197.50","25,992.00"
4,2025-10-29,"26,116.50","26,134.00",0.07%,"26,278.00","26,112.00"


In [7]:
df = pd.merge(df_Nifty, df_Gnifty, on='Date', how='inner')
display(df)

,Date,Close,High,Low,Open,Volume,RSI,MACD,MACD_Signal,EMA_50,SMA_20,StochRSI_K,StochRSI_D,GIFT_Open,GIFT_Close,Change(%),GIFT_High,GIFT_Low
0,2025-04-21,24125.550781,24189.550781,23903.650391,23949.150391,406100,NaN,0.000000,0.000000,24125.550781,NaN,NaN,NaN,"23,900.50","24,075.50",0.73%,"24,193.50","23,790.00"
1,2025-04-22,24167.250000,24242.599609,24072.000000,24185.400391,440900,NaN,3.326433,0.665287,24127.186045,NaN,NaN,NaN,"24,075.50","24,315.00",0.99%,"24,387.00","24,075.00"
2,2025-04-23,24328.949219,24359.300781,24119.949219,24357.599609,415000,NaN,18.793788,4.290987,24135.098326,NaN,NaN,NaN,"24,445.00","24,267.00",-0.73%,"24,445.00","24,128.50"
3,2025-04-24,24246.699219,24347.849609,24216.150391,24277.900391,358800,NaN,24.136659,8.260121,24139.474832,NaN,NaN,NaN,"24,280.00","24,519.50",0.99%,"24,879.00","24,223.00"
4,2025-04-25,24039.349609,24365.449219,23847.849609,24289.000000,387700,NaN,11.506897,8.909476,24135.548352,NaN,NaN,NaN,"24,513.50","24,220.00",-1.20%,"24,546.50","23,969.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2025-10-29,26053.900391,26097.849609,25960.300781,25982.000000,321900,82.726544,268.728626,208.777421,25245.831645,25419.345215,0.837457,0.845284,"26,116.50","26,134.00",0.07%,"26,278.00","26,112.00"
131,2025-10-30,25877.849609,26032.050781,25845.250000,25984.400391,257400,72.026127,261.044734,219.230884,25270.616663,25482.682715,0.758621,0.805434,"26,133.50","26,042.00",-0.35%,"26,197.50","25,992.00"
132,2025-10-31,25722.099609,25953.750000,25711.199219,25863.800781,334400,63.378740,239.625220,223.309751,25288.321877,25526.972656,0.617375,0.737818,"26,042.00","25,803.50",-0.92%,"26,113.00","25,803.50"
133,2025-11-03,25763.349609,25803.099609,25645.500000,25696.849609,23300,66.589251,223.403376,223.328476,25306.950415,25570.427637,0.471234,0.615744,"25,832.50","25,897.50",0.25%,"25,938.50","25,784.50"


In [9]:

# Convert string columns to numeric by removing commas in df
for col in ['GIFT_Open', 'GIFT_Close']:
    df[col] = df[col].astype(str).str.replace(',', '', regex=False)
    df[col] = pd.to_numeric(df[col])

feature_cols = ['Open','Close', 'RSI', 'MACD', 'MACD_Signal', 'EMA_50', 'SMA_20', 'StochRSI_K', 'StochRSI_D', 'GIFT_Open', 'GIFT_Close']

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[feature_cols])

# Prepare the input for prediction
x_input = np.array([scaled_data[-100:]])  
x_input = x_input.reshape(1, 100, 11)  

# Predict today's price
y_predicted_scaled = model.predict(x_input)

dummy_array = np.zeros(shape=(len(y_predicted_scaled), 11))

dummy_array[:, 0] = y_predicted_scaled[:, 0] # Predicted 'Close'
dummy_array[:, 1] = y_predicted_scaled[:, 1] # Predicted 'Open'

predicted_prices = scaler.inverse_transform(dummy_array)

# --- Display the results ---
print(f"Predicted Close Price for {now.date()}: {predicted_prices[0, 0]:.2f}")
print(f"Predicted Open Price for {now.date()}: {predicted_prices[0, 1]:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Predicted Close Price for 2025-11-05: 25672.14
Predicted Open Price for 2025-11-05: 25649.92
